In [1]:
pip install nlpaug

     |████████████████████████████████| 410 kB 8.7 MB/s 


In [2]:
pip install transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 6.5 MB 57.2 MB/s 
     |████████████████████████████████| 596 kB 74.3 MB/s 
     |████████████████████████████████| 895 kB 67.1 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install transformers[sentencepiece]

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [1]:
import torch
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Load test data

test_path = "/content/drive/MyDrive/GrkPos.csv"
df = pd.read_csv(test_path, encoding='utf8')
# encoding='cp1252 for SB_DataBase.csv 

In [4]:
len(df)

24705

In [ ]:
text = 'Can make it clear that even a fully person can get infected and spread virus then why they encouraging to take vaccine ? Why dont they disclose sero positivity in media ? Please spread love . Not fear .'
print(text)


Can make it clear that even a fully person can get infected and spread virus then why they encouraging to take vaccine ? Why dont they disclose sero positivity in media ? Please spread love . Not fear .


In [ ]:
## TEST THESE AUGMENTERS
#https://towardsdatascience.com/nlp-data-augmentation-using-transformers-89a44a993bab

In [59]:
from transformers import pipeline
import random

unmasker = pipeline('fill-mask', model='bert-base-cased')

input_text = " I had covid in the fall. It wasnt the worst thing I have ever experienced but Im really happy that the vaccine is available."

orig_text_list = input_text.split()
len_input = len(orig_text_list)
#Random index where we want to insert the word except at the start or end
rand_idx = random.randint(1,len_input-2)

new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
new_mask_sent = ' '.join(new_text_list)
print("Masked sentence->",new_mask_sent)
#I went to see a [Mask] movie in the theater

augmented_text_list = unmasker(new_mask_sent)
augmented_text = augmented_text_list[0]['sequence']
print("Augmented text->",augmented_text)
#I went to see a new movie in the theater

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Masked sentence-> I [MASK] had covid in the fall. It wasnt the worst thing I have ever experienced but Im really happy that the vaccine is available.
Augmented text-> I even had covid in the fall. It wasnt the worst thing I have ever experienced but Im really happy that the vaccine is available.


In [ ]:
df = df.iloc[3210:6256]

In [ ]:
#https://github.com/AdirthaBorgohain/BERT-Multilabel-Text-Classification/blob/main/notebooks/data_preparation.ipynb

In [ ]:
df

,tweet,polarity
0,"Yes, to bring to light the brutality of the po...",positive
1,Can make it clear that even a full-fledged per...,positive
2,If you suspect a side effect after a Covid-19 ...,positive
3,The development of some Covid - 19 in are good...,positive
4,I think you had no money to give your people a...,positive
...,...,...
3045,"""Widespread societal resistance to COVID-19 is...",negative
3046,This covid vaccine really got in the way!!!!!,negative
3047,"Yes, they have spread the most egregious in al...",negative
3048,"If y ""all had any sense and"" all was known, th...",negative


In [5]:
import pandas as pd
from tqdm import tqdm
import nlpaug.augmenter.word as naw

In [6]:
#Helsinki-NLP/opus-mt-ko-en
back_translation_aug = naw.BackTranslationAug(
    from_model_name='Helsinki-NLP/opus-mt-en-grk', 
    to_model_name='Helsinki-NLP/opus-mt-grk-en'
)
    
def generate_augmented_text(text):
    augmented_text = back_translation_aug.augment(text)
    return augmented_text

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

In [7]:
text = '''I had the disease in the fall. It wasn't the worst thing I have ever experienced but I'm really happy that the vaccine is available.'''

print("Original Text: ", text)
print("Augmented Text: ", generate_augmented_text(text))

Original Text:  I had the disease in the fall. It wasn't the worst thing I have ever experienced but I'm really happy that the vaccine is available.
Augmented Text:  I had the disease in the fall. It wasn't the worst thing I've ever experienced, but I'm really happy that the vaccine is available.


In [ ]:
#df = df.loc[df['polarity'] == 'positive']

In [7]:
df = df.dropna(subset=['tweet'])

In [8]:
len(df)

24705

In [9]:
df = df.drop_duplicates(subset=['tweet'])

In [10]:
len(df)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,tweet,polarity
0,0,0,0,0,"Yes, to bring to light the brutality of the po...",positive
1,1,1,1,1,Can make it clear that even a full-fledged per...,positive
2,2,2,2,2,If you suspect a side effect after a Covid-19 ...,positive
3,3,3,3,3,The development of some Covid - 19 in are good...,positive
4,4,4,4,4,I think you had no money to give your people a...,positive
...,...,...,...,...,...,...
24700,24898,17664,9317,6020,After vaccination after vaccination after you ...,positive
24701,24899,17665,9320,6026,To turn to pull for the emblems and kin COVID-...,positive
24702,24900,17666,9321,6027,The more true a man with a heart can't see or ...,positive
24703,24901,17667,9323,6035,Nobody needs to admit nobody needs to admit no...,positive


In [11]:
def augment_category(category_label: str, df: pd.DataFrame):
    indexes = df[df['polarity'] == 'negative'].index

    for idx in tqdm(indexes):
        text = df.loc[idx]['tweet']
        df = df.append(df.loc[[idx]].assign(**{'tweet': generate_augmented_text(text)}), 
                  ignore_index=True)
    return df

In [12]:
df.reset_index()

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,tweet,polarity
0,0,0,0,0,0,"Yes, to bring to light the brutality of the po...",positive
1,1,1,1,1,1,Can make it clear that even a full-fledged per...,positive
2,2,2,2,2,2,If you suspect a side effect after a Covid-19 ...,positive
3,3,3,3,3,3,The development of some Covid - 19 in are good...,positive
4,4,4,4,4,4,I think you had no money to give your people a...,positive
...,...,...,...,...,...,...,...
24700,24700,24898,17664,9317,6020,After vaccination after vaccination after you ...,positive
24701,24701,24899,17665,9320,6026,To turn to pull for the emblems and kin COVID-...,positive
24702,24702,24900,17666,9321,6027,The more true a man with a heart can't see or ...,positive
24703,24703,24901,17667,9323,6035,Nobody needs to admit nobody needs to admit no...,positive


In [13]:
categories_to_augment = ['polarity']

for category in categories_to_augment:
    df = augment_category(category, df=df)

 16%|█▋        | 1660/10069 [2:35:52<13:09:35,  5.63s/it]


KeyboardInterrupt: ignored

In [16]:
 df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,tweet,polarity
0,0,0,0,0,"Yes, to bring to light the brutality of the po...",positive
1,1,1,1,1,Can make it clear that even a full-fledged per...,positive
2,2,2,2,2,If you suspect a side effect after a Covid-19 ...,positive
3,3,3,3,3,The development of some Covid - 19 in are good...,positive
4,4,4,4,4,I think you had no money to give your people a...,positive
...,...,...,...,...,...,...
33853,13779,13779,9374,1712,Bill shows VOCID vaccines the educational pests.,negative
33854,13780,13780,9375,1713,I would never let them give me a chance with V...,negative
33855,13781,13781,9377,1717,"Bill met him several times, even if he had alr...",negative
33856,13782,13782,9378,1721,Rover really is a virologist... and if Covid a...,negative


In [17]:
df = df.drop_duplicates(subset=['tweet'])

In [18]:
df = df.drop_duplicates(subset=['tweet'])

In [19]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,tweet,polarity
0,0,0,0,0,"Yes, to bring to light the brutality of the po...",positive
1,1,1,1,1,Can make it clear that even a full-fledged per...,positive
2,2,2,2,2,If you suspect a side effect after a Covid-19 ...,positive
3,3,3,3,3,The development of some Covid - 19 in are good...,positive
4,4,4,4,4,I think you had no money to give your people a...,positive
...,...,...,...,...,...,...
33851,13777,13777,9372,1710,Tell Natt all the poison vaccines and it won't...,negative
33853,13779,13779,9374,1712,Bill shows VOCID vaccines the educational pests.,negative
33854,13780,13780,9375,1713,I would never let them give me a chance with V...,negative
33856,13782,13782,9378,1721,Rover really is a virologist... and if Covid a...,negative


In [20]:
df.to_csv('GrkNeg.csv')
!cp GrkNeg.csv "drive/MyDrive/" 

In [41]:
df

,Unnamed: 0,tweet,polarity
0,0,"Yes, to bring to light the brutality of the po...",positive
1,1,Can make it clear that even a full-fledged per...,positive
2,2,If you suspect a side effect after a Covid-19 ...,positive
3,3,The development of some Covid - 19 in are good...,positive
4,4,I think you had no money to give your people a...,positive
...,...,...,...
9375,1713,Never in a million years would I allow them to...,negative
9376,1714,Never believe what they say about COVID-19.,negative
9377,1717,"Bill has met him many times, even though he ha...",negative
9378,1721,Rover Really. Virologist... what if Covid alre...,negative
